In [ ]:
!pip uninstall tensorflow-io -y
!pip install --no-deps tensorflow-io

In [75]:
from keras.applications import VGG16
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from numpy.random import seed
import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [ ]:
path_train="../input/rca-lca/train"
path_test="../input/rca-lca/Test"

In [ ]:
path_all="../input/rca-lca"

In [ ]:
batch_size=32
nb_epochs=50

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,       
        rotation_range = 20 , # Rotate images randomly within the range of -20 to +20 degrees
        width_shift_range = 0.1 , # Randomly shift images horizontally within the range of -10% to +10%
        height_shift_range = 0.1 , # Randomly shift images vertically within the range of -10% to +10%
        zoom_range = 0.2 , # Randomly zoom images within the range of 80% to 120%
        horizontal_flip = True , # Randomly flip images horizontally
        vertical_flip = True , # Randomly flip images vertically
        validation_split=0.2
    )

In [ ]:
train_generator = datagen.flow_from_directory(path_all,
                                                subset='training',
                                                target_size=(128, 128),
                                                batch_size=28, #batch_size,
                                                validation=0.2,
                                                class_mode='binary')

In [ ]:
valid_generator = datagen.flow_from_directory(
    path_all, # same directory as training data
    subset='validation',
    target_size=(128, 128),
    batch_size=28,
    class_mode='binary' # Type of labels (e.g., binary or categorical)
    )

In [ ]:
train_generator.image_shape

In [ ]:
train_generator.target_size

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(path_test,
                                                  target_size=(128, 128),
                                                  batch_size=28,
                                                  class_mode='binary')

In [ ]:
dual_stream_model="best_DS_Model.h5"
checkpoint = ModelCheckpoint(dual_stream_model,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True, 
                             mode='auto') 
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [76]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model


In [73]:
def drawModel(model_2streams):
    # Tracer le modèle
    plot_model(model_2streams, to_file='model_2streams.png', 
               show_shapes=True, 
               show_layer_names=True)
    
    image = plt.imread('model_2streams.png')
    
    plt.figure(figsize=(20, 40))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

=========== Dual-Dense ===============

In [ ]:
input_shape = (128, 128, 3)

In [ ]:
import tensorflow as tf

In [ ]:
input_A = tf.keras.layers.Input(shape=input_shape, name="left")
hidden1A = tf.keras.layers.Dense(16, activation="relu")(input_A)
hidden2A = tf.keras.layers.Dense(32, activation="relu")(hidden1A)
hidden3A = tf.keras.layers.Dense(64, activation="relu")(hidden2A)
hidden4A = tf.keras.layers.Dense(32, activation="relu")(hidden3A)


input_B = tf.keras.layers.Input(shape=input_shape, name="right")
hidden1B = tf.keras.layers.Dense(30, activation="relu")(input_B)
hidden2B = tf.keras.layers.Dense(30, activation="relu")(hidden1B)

concat = tf.keras.layers.concatenate([hidden4A, hidden2B])
flatten = tf.keras.layers.Flatten()(concat)

output = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(flatten)
model = tf.keras.Model(inputs=[input_A, input_B], outputs=[output])


In [ ]:
drawModel(model)

In [ ]:
labels=train_generator.next()[1]
labels = tf.expand_dims(labels, axis=-1)
labels = tf.expand_dims(labels, axis=-1)
labels = tf.expand_dims(labels, axis=-1)
labels.shape

In [ ]:
batch_size = len(train_generator)
x_train, y_train = next(train_generator)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit([x_train, x_train ], y_train,
            steps_per_epoch=1330/batch_size,
            epochs = 50
         )

In [ ]:
x_test, y_test = next(test_generator)

In [ ]:
#test_labels=train_generator.next()[1]
#y_test = tf.expand_dims(y_test, axis=-1)
#y_test = tf.expand_dims(y_test, axis=-1)
#y_test = tf.expand_dims(y_test, axis=-1)

In [ ]:
Y_pred = model.predict([x_test, x_test])
y_pred_classes = (Y_pred > 0.5).astype(int)  # Convert probabilities to classes (0 or 1)

cm = confusion_matrix(y_test, y_pred_classes)

print(cm)

----------------- dual cnn classique ------------------

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate,add
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size=28
nb_epochs=50
input_shape = (128, 128, 3)

In [ ]:
def myCNN1():
        model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
    ])
        return model

In [ ]:
def myCNN2():
        model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten()
    ])
        return model

In [ ]:
stream1 = myCNN1()
stream1_output = stream1.output
#stream1_output = Flatten()(stream1_output)


stream2 = myCNN2()
stream2_output = stream2.output
#stream2_output = Flatten()(stream2_output)

# Concatenate the outputs of both streams
merged = concatenate([stream1_output, stream2_output])

output = Dense(16, activation='relu')(merged)
# Add a dense layer for classification
output = Dense(1, activation='sigmoid')(output)

# Create the dual-stream model
model_2streams = Model(inputs=[stream1.input,stream2.input], outputs=output)


In [ ]:
name_model="best_2streams_CNN_Model.h5"
checkpoint = ModelCheckpoint(model_2streams,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True, 
                             mode='auto') 
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [ ]:
model_2streams.compile(optimizer='adam', 
                       loss='binary_crossentropy', 
                       metrics=['accuracy']
                      )

In [ ]:
plot_model(model_2streams, to_file='model_2streams_CNN.png')

In [ ]:
drawModel(model_2streams)

In [ ]:
steps_per_epoch=1330//batch_size
validation_steps=151//batch_size
epochs=50

In [ ]:
y_train=train_generator.next()[1]
y_train = tf.expand_dims(y_train, axis=-1)
#y_train = tf.expand_dims(y_train, axis=-1)
#y_train = tf.expand_dims(y_train, axis=-1)

In [ ]:
x_train=train_generator.next()[0]
y_train=train_generator.next()[1]

In [ ]:
x_train_stream1 = train_generator.next()[0]
x_train_stream2 = train_generator.next()[0]

In [ ]:
y_valid=valid_generator.next()[1]
y_valid = tf.expand_dims(y_valid, axis=-1)
#y_valid = tf.expand_dims(y_valid, axis=-1)
#y_valid = tf.expand_dims(y_valid, axis=-1)

In [ ]:
x_valid_stream1 = valid_generator.next()[0]
x_valid_stream2 = valid_generator.next()[0]

In [ ]:
y_valid.shape

In [ ]:
history=model_2streams.fit([x_train_stream1, x_train_stream2],y_train,
                        steps_per_epoch=1330/batch_size,
                        epochs = epochs,
                        validation_data = ([x_valid_stream1,x_valid_stream2],y_valid),   
                        validation_steps = 151/batch_size,
                        use_multiprocessing=True,
                        shuffle=False)

In [ ]:
x_test, y_test = next(test_generator)

In [ ]:
Y_pred = model_2streams.predict([x_test, x_test])
y_pred_classes = (Y_pred > 0.5).astype(int)  
# Convert probabilities to classes (0 or 1)
cm = confusion_matrix(y_test, y_pred_classes)

print(cm)

=========== Dual AngioKey =============

In [ ]:
def angioKey():
    
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten() ])
    
    return model

In [ ]:
stream1 = angioKey()
stream1_output = stream1.output
#stream1_output = Flatten()(stream1_output)


stream2 = angioKey()
stream2_output = stream2.output
#stream2_output = Flatten()(stream2_output)

# Concatenate the outputs of both streams
merged = concatenate([stream1_output, stream2_output])

output = Dense(16, activation='relu')(merged)
# Add a dense layer for classification
output = Dense(1, activation='sigmoid')(output)

# Create the dual-stream model
dual_angiokey = Model(inputs=[stream1.input,stream2.input], outputs=output)


In [ ]:
dual_angiokey.compile(optimizer='adam', 
                       loss='binary_crossentropy', 
                       metrics=['accuracy']
                      )

In [ ]:
drawModel(dual_angiokey)

In [ ]:
angiokey_history=dual_angiokey.fit([x_train_stream1, x_train_stream2],y_train,
                        steps_per_epoch=1330/batch_size,
                        epochs = epochs,
                        validation_data = ([x_valid_stream1,x_valid_stream2],y_valid),   
                        validation_steps = 151/batch_size,
                        use_multiprocessing=True,
                        shuffle=False)

In [ ]:
Y_pred = dual_angiokey.predict([x_test, x_test])
y_pred_classes = (Y_pred > 0.5).astype(int)  
# Convert probabilities to classes (0 or 1)
cm = confusion_matrix(y_test, y_pred_classes)

print(cm)

--------------- dual inception-vgg ------------------------

In [ ]:
from tensorflow.keras.layers import Input, Dense, concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3, VGG19
import tensorflow as tf

input_shape = (128, 128, 3)

# Create the Inception model
inception_input = Input(shape=input_shape)
inception_model = InceptionV3(input_tensor=inception_input, 
                              weights='imagenet', 
                              include_top=False)
inception_model=inception_model.layers[-1]
#inception_output = GlobalAveragePooling2D()(inception_model.output)
inception_output = Flatten()(inception_model.output)

# Create the ResNet-50 model
vgg19_input = Input(shape=input_shape)
vgg19_model = VGG19(input_tensor=vgg19_input, weights='imagenet', include_top=False)
vgg19_model = vgg19_model.layers[-1]
#vgg19_output = GlobalAveragePooling2D()(vgg19_model.output)
vgg19_output = Flatten()(vgg19_model.output)

# Concatenate the outputs of the two models
merged = concatenate([inception_output, vgg19_output])

# Add a fully connected layer
output = Dense(1, activation='sigmoid')(merged)

# Create the combined model
#model_2streams = Model(inputs=[inception_input, vgg19_input], outputs=output)
model_vgg_incept = Model(inputs=[inception_input, vgg19_input], outputs=output)


In [ ]:
drawModel(model_vgg_incept)

In [ ]:
# Compiler le modèle
model_vgg_incept.compile(optimizer='adam', 
                       loss='binary_crossentropy', 
                       metrics=['accuracy'])

In [ ]:
train_generator_1, train_generator_2=train_generator , train_generator

In [ ]:
vgg_incept_model="best_vgg_incept_Model.h5"
checkpoint = ModelCheckpoint(vgg_incept_model,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True, 
                             mode='auto') 
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [ ]:
history_vgg_incept = model_vgg_incept.fit([x_train_stream1, x_train_stream2],y_train,
                        steps_per_epoch=1330/batch_size,
                        epochs = epochs,
                        validation_data = ([x_valid_stream1,x_valid_stream2],y_valid),   
                        validation_steps = 151/batch_size,
                        use_multiprocessing=True,
                        shuffle=False,
                        callbacks=[early_stopping,checkpoint])

In [ ]:
Y_pred = model_vgg_incept.predict([x_test, x_test])
y_pred_classes = (Y_pred > 0.5).astype(int)  
# Convert probabilities to classes (0 or 1)
cm = confusion_matrix(y_test, y_pred_classes)

print(cm)

================Dual stream Dense-CNN ================

In [1]:
path_metadata='../input/metadata.csv'

In [2]:
import pandas as pd
df = pd.read_csv(path_metadata, sep =',',  header =0)
df.head()

,fileName,NamePatient,PositionerPrimaryAngle,PositionerSecondaryAngle,DistanceSourceToDetector,DistanceDetectorToPatient,Label
0,FRIJI_ABDELFATTEH_CORO_23905-CORO_2020_0001-fr...,FRIJI_ABDELFATTEH_CORO_23905,1.4,-23.9,1102,797.061368,LCA
1,GAMMOUDI_BECHIR_CORO_24350-CORO_2020_0001-fram...,GAMMOUDI_BECHIR_CORO_24350,-1.8,-30.9,1097,707.012615,LCA
2,GAFSI_GHZALA_CORO_25467-CORO_2020_0004-frame_0...,GAFSI_GHZALA_CORO_25467,-32.7,28.4,1200,851.978788,LCA
3,GAFSI_GHZALA_CORO_25467-CORO_2020_0002-frame_0...,GAFSI_GHZALA_CORO_25467,-24.6,-37.1,1200,853.897363,LCA
4,GAFSI_GHZALA_CORO_25467-CORO_2020_0006-frame_0...,GAFSI_GHZALA_CORO_25467,25.2,18.6,1200,842.435323,LCA


In [3]:
df.shape

(1482, 7)

In [4]:
df['DistanceDetectorToPatient'].isna().sum()

20

In [5]:
df.dropna(inplace=True)

In [6]:
df.drop('NamePatient', inplace=True, axis=1)

In [7]:
df.shape

(1462, 6)

In [8]:
def create_mlp(dim):
        # define our MLP network
        model = Sequential()
        model.add(Dense(8, input_dim=dim, activation="relu"))
        model.add(Dense(4, activation="relu"))
        # check to see if the regression node should be added
        
        return model

In [9]:
def create_cnn(filters=(16, 32, 64)):
        # initialize the input shape and channel dimension, assuming
        # TensorFlow/channels-last ordering
        inputShape = (512, 512, 3)
        chanDim = -1
        # define the model input
        inputs = Input(shape=inputShape)
        # loop over the number of filters
        for (i, f) in enumerate(filters):
            # if this is the first CONV layer then set the input
            # appropriately
            if i == 0:
                x = inputs
            # CONV => RELU => BN => POOL
            x = Conv2D(f, (3, 3), padding="same")(x)
            x = Activation("relu")(x)
            x = BatchNormalization(axis=chanDim)(x)
            x = MaxPooling2D(pool_size=(2, 2))(x)
            
        # flatten the volume, then FC => RELU => BN => DROPOUT
        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)
        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP
        x = Dense(4)(x)
        x = Activation("relu")(x)
        
        # construct the CNN
        model = Model(inputs, x)
        # return the CNN
        return model

In [10]:
import glob
import os
import numpy as np
import cv2

In [24]:
liste_rep=os.listdir(path_all)
liste_all=[]
for rep in liste_rep :
    path_rep=join(path_all,rep)
    liste_img=os.listdir(path_rep)
    liste_all.extend(liste_img)
print(len(liste_all))

1482


In [25]:
liste_all[0]

'JMAL_WAHID_CORO_24605-CORO_2020_0004-frame_00012.jpg'

In [12]:
imageNames = df.fileName
targets=df.Label
len(imageNames)

1462

In [13]:
def load_images():
        images = []  
        problems=[]
        
        for i in range(len(imageNames)) :
            try :
                #print(i," ",imageNames[i],' ',targets[i])
                nameImage=imageNames[i]
                if targets[i]=="LCA":
                    pathImage=os.path.join(path_all,"LCA", nameImage)
                else:
                    pathImage=os.path.join(path_all,"RCA", nameImage)
                 
                image = cv2.imread(pathImage)
                #image = cv2.resize(image, (128, 128))
                images.append(image)
                
            except:
                problems.append(nameImage)
                pass
            
        # return our set of images
        return images ,problems   

In [21]:
images,problems = load_images()
print(len(images)," loaded images...")
print(len(problems)," problems...")

1442  loaded images...
20  problems...


In [17]:
indexProb = df[df['fileName'].isin(problems)].index
df.drop(indexProb, inplace=True)

In [18]:
df.shape

(1444, 6)

In [19]:
len(images)

1442

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.drop('fileName', inplace=True, axis=1)

In [ ]:
X = df.iloc[:,:-1]
Y= df.iloc[:,-1]

In [ ]:
X=X.values
Y=Y.values

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

Y[:] = label.fit_transform(Y[:])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)

In [ ]:
len(images)

In [ ]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainAttrX & testAttrX sont des dataframes

In [ ]:
trainY = np.asarray(trainAttrX.Label)
trainAttrX = np.asarray(trainAttrX[:-1])
testY = np.asarray(testAttrX.Label)
testAttrX = np.asarray(testAttrX[:-1])

In [ ]:
"""trainY=trainAttrX.Label.values
trainAttrX=trainAttrX[:-1].values

testY=testAttrX.Label.values
testAttrX=testAttrX[:-1].values"""

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
trainAttrX.shape[1]

In [ ]:
# create the MLP and CNN models
mlp = create_mlp(trainAttrX.shape[1])
cnn = create_cnn((128, 128, 3))
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [ ]:
#opt = Adam(lr=1e-3, decay=1e-3 / 200)
#model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
model.compile(optimizer='adam', 
                       loss='binary_crossentropy', 
                       metrics=['accuracy'])

In [ ]:
drawModel(model)

In [ ]:
hist=model.fit([trainImagesX,trainAttrX], trainY,
        validation_data=([testImagesX,testAttrX], testY),
        epochs=200, batch_size=8)

In [ ]:
preds = model.predict([testAttrX, testImagesX])

In [ ]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)